In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('us_dummy.csv')


data['covid_2020_2021'] = data['year'].isin([2020, 2021]).astype(int)


data['gdp_centered'] = data['gdp'] - data['gdp'].mean()

data['ln_net_migration'] = np.log(data['net_migration'] + 100)  
data['ln_gdp_centered'] = np.log(data['gdp_centered'].abs() + 1) * np.sign(data['gdp_centered'])
data['ln_unemp'] = np.log(data['unemp'] + 1)
data['ln_polstab'] = np.log(data['polstab'] + 2)  


X = data[['ln_gdp_centered', 'ln_unemp', 'ln_polstab', 'covid_2020_2021']]
print("Correlation Matrix:")
print(X.corr())

print("NaN/Inf Check Before Model:")
print(X.isna().sum())
print(X.isin([np.inf, -np.inf]).sum())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)


vif_data = pd.DataFrame()
vif_data['variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("VIF Results:")
print(vif_data)

y = data['ln_net_migration']

X_scaled_with_const = sm.add_constant(X_scaled)


model = sm.GLSAR(y, X_scaled_with_const, rho=1).fit()
print("Prais-Winsten Model Summary:")
print(model.summary())


ridge = Ridge(alpha=1.0)  
ridge.fit(X_scaled, y)
print("Ridge Regression Coefficients:")
for var, coef in zip(X.columns, ridge.coef_):
    print(f"{var}: {coef:.4f}")
print(f"Ridge Intercept: {ridge.intercept_:.4f}")


predictions = model.predict(X_scaled_with_const)
data['predicted_net_migration'] = np.exp(predictions) - 100 

# Save predictions
# data.to_csv('net_migration_predictions_usa_improved.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


data = pd.read_csv('us_dummy.csv')


data['covid_2020_2021'] = data['year'].isin([2020, 2021]).astype(int)


data['gdp_centered'] = data['gdp'] - data['gdp'].mean()


data['ln_net_migration'] = np.log(data['net_migration'] + 100)
data['ln_gdp_centered'] = np.log(data['gdp_centered'].abs() + 1) * np.sign(data['gdp_centered'])

data['gdp_covid_interaction'] = data['ln_gdp_centered'] * data['covid_2020_2021']


X = data[['ln_gdp_centered', 'covid_2020_2021', 'gdp_covid_interaction']]
y = data['ln_net_migration']


print("Correlation Matrix:")
print(X.corr())


print("NaN/Inf Check Before Model:")
print(X.isna().sum())
print(X.isin([np.inf, -np.inf]).sum())


vif_data = pd.DataFrame()
vif_data['variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("VIF Results:")
print(vif_data)


X = sm.add_constant(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = sm.GLSAR(y_train, X_train, rho=1).fit()
print("Prais-Winsten Model Summary:")
print(model.summary())

residuals = model.resid
std_residuals = residuals / residuals.std()
print("Standardized Residuals Check (absolute values > 3):")
print(std_residuals[abs(std_residuals) > 3])

predictions_test = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
print(f"Test RMSE: {rmse:.4f}")

predictions = model.predict(X)
data['predicted_net_migration'] = np.exp(predictions) - 100

data.to_csv('US_prais.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os


data = pd.read_csv('migrationdata.csv')


if not os.path.exists('country_results'):
    os.makedirs('country_results')


countries = data['country'].unique()

for country in countries:
    print(f"\nProcessing {country}...\n")
    

    country_data = data[data['country'] == country].copy()
    

    country_data['covid_2020_2021'] = country_data['year'].isin([2020, 2021]).astype(int)
    

    country_data['gdp_centered'] = country_data['gdp'] - country_data['gdp'].mean()
    


    country_data['ln_net_migration'] = np.log(country_data['net_migration'] + 100)
    country_data['ln_gdp_centered'] = np.log(country_data['gdp_centered'].abs() + 1) * np.sign(country_data['gdp_centered'])
    

    country_data['gdp_covid_interaction'] = country_data['ln_gdp_centered'] * country_data['covid_2020_2021']
    
 
    X = country_data[['ln_gdp_centered', 'covid_2020_2021', 'gdp_covid_interaction']]
    y = country_data['ln_net_migration']
    

    print(f"Correlation Matrix for {country}:")
    print(X.corr())
    

    print(f"\nNaN/Inf Check Before Model for {country}:")
    print(X.isna().sum())
    print(X.isin([np.inf, -np.inf]).sum())
    

    vif_data = pd.DataFrame()
    vif_data['variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(f"\nVIF Results for {country}:")
    print(vif_data)
    

    X = sm.add_constant(X)
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

    try:
        model = sm.GLSAR(y_train, X_train, rho=1).fit()
        print(f"\nPrais-Winsten Model Summary for {country}:")
        print(model.summary())
        

        residuals = model.resid
        std_residuals = residuals / residuals.std()
        print(f"\nStandardized Residuals Check for {country} (absolute values > 3):")
        print(std_residuals[abs(std_residuals) > 3])
        

        predictions_test = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
        print(f"\nTest RMSE for {country}: {rmse:.4f}")
        

        predictions = model.predict(X)
        country_data['predicted_net_migration'] = np.exp(predictions) - 100
        

        output_file = f'./model_outputs/{country.replace(" ", "_")}_prais.csv'
        country_data.to_csv(output_file, index=False)
        print(f"Predictions saved to {output_file}")
        
    except Exception as e:
        print(f"Error fitting model for {country}: {str(e)}")

print("\nProcessing complete for all countries.")

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os


data = pd.read_csv('migrationdata.csv')


if not os.path.exists('country_results'):
    os.makedirs('country_results')


countries = data['country'].unique()


log_file = open('country_results/error_log.txt', 'w')

for country in countries:
    print(f"\nProcessing {country}...\n")
    

    country_data = data[data['country'] == country].copy()
    

    if len(country_data) < 10:
        error_msg = f"Error for {country}: Insufficient data points ({len(country_data)} < 10)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    

    if country_data[['gdp', 'net_migration']].isna().any().any():
        error_msg = f"Error for {country}: Missing values in gdp or net_migration\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    country_data['covid_2020_2021'] = country_data['year'].isin([2020, 2021]).astype(int)

    country_data['gdp_centered'] = country_data['gdp'] - country_data['gdp'].mean()

    min_net_migration = country_data['net_migration'].min()
    shift = abs(min_net_migration) + 100 if min_net_migration <= 0 else 100
    country_data['ln_net_migration'] = np.log(country_data['net_migration'] + shift)
    country_data['ln_gdp_centered'] = np.log(country_data['gdp_centered'].abs() + 1) * np.sign(country_data['gdp_centered'])
    

    X = country_data[['ln_gdp_centered', 'covid_2020_2021']]
    y = country_data['ln_net_migration']
    

    print(f"Correlation Matrix for {country}:")
    print(X.corr())
    

    print(f"\nNaN/Inf Check Before Model for {country}:")
    print(X.isna().sum())
    print(X.isin([np.inf, -np.inf]).sum())
    

    vif_data = pd.DataFrame()
    vif_data['variable'] = X.columns
    try:
        vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        print(f"\nVIF Results for {country}:")
        print(vif_data)
    except np.linalg.LinAlgError:
        error_msg = f"Error for {country}: VIF calculation failed (possible singular matrix)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    

    X = sm.add_constant(X)
    

    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    except ValueError as e:
        error_msg = f"Error for {country}: Train-test split failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    

    try:
        model = sm.GLSAR(y_train, X_train, rho=1).fit()
        model_type = "Prais-Winsten"
    except (np.linalg.LinAlgError, ValueError) as e:
        error_msg = f"Warning for {country}: Prais-Winsten failed ({str(e)}), falling back to OLS\n"
        print(error_msg)
        log_file.write(error_msg)
        model = sm.OLS(y_train, X_train).fit()
        model_type = "OLS"
    
    print(f"\n{model_type} Model Summary for {country}:")
    print(model.summary())
    

    residuals = model.resid
    std_residuals = residuals / residuals.std()
    print(f"\nStandardized Residuals Check for {country} (absolute values > 3):")
    print(std_residuals[abs(std_residuals) > 3])

    predictions_test = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
    print(f"\nTest RMSE for {country}: {rmse:.4f}")
    

    predictions = model.predict(X)
    country_data['predicted_net_migration'] = np.exp(predictions) - shift
    

    output_file = f'country_results/{country.replace(" ", "_")}_prais.csv'
    country_data.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

log_file.close()
print("\nProcessing complete for all countries. Check 'country_results/error_log.txt' for details on any errors.")

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
import matplotlib.pyplot as plt

data = pd.read_csv('migrationdata.csv')

if not os.path.exists('country_results'):
    os.makedirs('country_results')


log_file = open('country_results/error_log.txt', 'w')

data = data.sort_values(['country', 'year'])  
data['covid_2020_2021'] = data['year'].isin([2020, 2021]).astype(int)

data['gdp_centered'] = data.groupby('country')['gdp'].transform(lambda x: x - x.mean())
data['ln_gdp_centered'] = np.log(data['gdp_centered'].abs() + 1) * np.sign(data['gdp_centered'])
data['ln_gdp_centered_sq'] = data['ln_gdp_centered'] ** 2 
data['ln_gini'] = np.log(data['gini'] + 1) 
data['ln_unemp'] = np.log(data['unemp'] + 1)  
data['polstab_centered'] = data.groupby('country')['polstab'].transform(lambda x: x - x.mean())

data['ln_gdp_centered_lag1'] = data.groupby('country')['ln_gdp_centered'].shift(1)
data['polstab_centered_lag1'] = data.groupby('country')['polstab_centered'].shift(1)

min_net_migration = data['net_migration'].min()
shift = abs(min_net_migration) + 1 if min_net_migration <= 0 else 1
data['ln_net_migration'] = np.log(data['net_migration'] + shift)

data = data.dropna()

countries = data['country'].unique()

for country in countries:
    print(f"\nProcessing {country}...\n")

    country_data = data[data['country'] == country].copy()

    if len(country_data) < 10:
        error_msg = f"Error for {country}: Insufficient data points ({len(country_data)} < 10)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    if country_data[['ln_net_migration', 'ln_gdp_centered', 'ln_gdp_centered_sq', 'ln_gini', 
                     'ln_unemp', 'polstab_centered', 'ln_gdp_centered_lag1', 
                     'polstab_centered_lag1', 'covid_2020_2021']].isna().any().any():
        error_msg = f"Error for {country}: Missing values in predictors or response\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    X = country_data[['ln_gdp_centered', 'ln_gdp_centered_sq', 'ln_gini', 'ln_unemp', 
                      'polstab_centered', 'ln_gdp_centered_lag1', 'polstab_centered_lag1', 
                      'covid_2020_2021']]
    y = country_data['ln_net_migration']

    print(f"Correlation Matrix for {country}:")
    print(X.corr())

    print(f"\nNaN/Inf Check Before Model for {country}:")
    print(X.isna().sum())
    print(X.isin([np.inf, -np.inf]).sum())

    vif_data = pd.DataFrame()
    vif_data['variable'] = X.columns
    try:
        vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        print(f"\nVIF Results for {country}:")
        print(vif_data)
        if (vif_data['VIF'] > 10).any():
            error_msg = f"Warning for {country}: High VIF detected, potential multicollinearity\n"
            print(error_msg)
            log_file.write(error_msg)
    except np.linalg.LinAlgError:
        error_msg = f"Error for {country}: VIF calculation failed (possible singular matrix)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    X = sm.add_constant(X)

    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    except ValueError as e:
        error_msg = f"Error for {country}: Train-test split failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    try:
        model = sm.GLSAR(y_train, X_train, rho=1).fit(cov_type='HC3') 
        model_type = "Prais-Winsten"
    except (np.linalg.LinAlgError, ValueError) as e:
        error_msg = f"Warning for {country}: Prais-Winsten failed ({str(e)}), falling back to OLS\n"
        print(error_msg)
        log_file.write(error_msg)
        model = sm.OLS(y_train, X_train).fit(cov_type='HC3')
        model_type = "OLS"
    
    print(f"\n{model_type} Model Summary for {country}:")
    print(model.summary())

    residuals = model.resid
    std_residuals = residuals / residuals.std()
    print(f"\nStandardized Residuals Check for {country} (absolute values > 3):")
    print(std_residuals[abs(std_residuals) > 3])

    predictions_test = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
    print(f"\nTest RMSE for {country}: {rmse:.4f}")

    predictions = model.predict(X)
    country_data['predicted_net_migration'] = np.exp(predictions) - shift
    
    # save predictions to CSV
    output = f'country_results/{country.replace(" ", "_")}_prais.csv'
    # country_data.to_csv(output_file, index=False)
    # print(f"Predictions saved to {output_file}")
    plt.scatter(predictions_test, y_test)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Predicted vs Actual for {country}')
    plt.show()

log_file.close()
print("\nProcessing complete for all countries. Check 'country_results/error_log.txt' for details on any errors.")

In [ ]:
from linearmodels.panel import PanelOLS
data_panel = data.set_index(['country', 'year'])
model = PanelOLS(data_panel['ln_net_migration'], 
                 data_panel[['ln_gdp_centered', 'ln_gdp_centered_sq', 'ln_gini', 
                             'ln_unemp', 'polstab_centered', 'ln_gdp_centered_lag1', 
                             'polstab_centered_lag1', 'covid_2020_2021']], 
                 entity_effects=True).fit(cov_type='robust')
print(model)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
import matplotlib.pyplot as plt

data = pd.read_csv('migrationdata.csv')

if not os.path.exists('country_results'):
    os.makedirs('country_results')

log_file = open('country_results/error_log.txt', 'w')

data = data.sort_values(['country', 'year'])  
data['covid_2020_2021'] = data['year'].isin([2020, 2021]).astype(int)

data['gdp_centered'] = data.groupby('country')['gdp'].transform(lambda x: x - x.mean())
data['ln_gdp_centered'] = np.log(data['gdp_centered'].abs() + 1) * np.sign(data['gdp_centered'])
data['ln_gdp_centered_sq'] = data['ln_gdp_centered'] ** 2  
data['ln_gini'] = np.log(data['gini'] + 1) 
data['ln_unemp'] = np.log(data['unemp'] + 1)  
data['polstab_centered'] = data.groupby('country')['polstab'].transform(lambda x: x - x.mean())

data['ln_gdp_centered_lag1'] = data.groupby('country')['ln_gdp_centered'].shift(1)
data['polstab_centered_lag1'] = data.groupby('country')['polstab_centered'].shift(1)

min_net_migration = data['net_migration'].min()
shift = abs(min_net_migration) + 1 if min_net_migration <= 0 else 1
data['ln_net_migration'] = np.log(data['net_migration'] + shift)

data = data.dropna()

countries = data['country'].unique()

for country in countries:
    print(f"\nProcessing {country}...\n")

    country_data = data[data['country'] == country].copy()

    if len(country_data) < 10:
        error_msg = f"Error for {country}: Insufficient data points ({len(country_data)} < 10)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    predictors = ['ln_net_migration', 'ln_gdp_centered', 'ln_gdp_centered_sq', 'ln_gini', 
                  'ln_unemp', 'polstab_centered', 'ln_gdp_centered_lag1', 
                  'polstab_centered_lag1', 'covid_2020_2021']
    if country_data[predictors].isna().any().any():
        error_msg = f"Error for {country}: Missing values in predictors or response\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    X = country_data[['ln_gdp_centered', 'ln_gdp_centered_sq', 'ln_gini', 'ln_unemp', 
                      'polstab_centered', 'ln_gdp_centered_lag1', 'polstab_centered_lag1', 
                      'covid_2020_2021']]
    y = country_data['ln_net_migration']

    print(f"Correlation Matrix for {country}:")
    print(X.corr())

    print(f"\nNaN/Inf Check Before Model for {country}:")
    print(X.isna().sum())
    print(X.isin([np.inf, -np.inf]).sum())

    vif_data = pd.DataFrame()
    vif_data['variable'] = X.columns
    try:
        vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        print(f"\nVIF Results for {country}:")
        print(vif_data)
        if (vif_data['VIF'] > 10).any():
            error_msg = f"Warning for {country}: High VIF detected, potential multicollinearity\n"
            print(error_msg)
            log_file.write(error_msg)
    except np.linalg.LinAlgError:
        error_msg = f"Error for {country}: VIF calculation failed (possible singular matrix)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue

    X = sm.add_constant(X)

    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    except ValueError as e:
        error_msg = f"Error for {country}: Train-test split failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
  
    models = {}

    try:
        model_ols = sm.OLS(y_train, X_train).fit(cov_type='HC3')
        models['OLS'] = {'model': model_ols, 'type': 'OLS'}
        print(f"\nOLS Model Summary for {country}:")
        print(model_ols.summary())

        residuals = model_ols.resid
        std_residuals = residuals / residuals.std()
        print(f"\nStandardized Residuals Check for OLS in {country} (absolute values > 3):")
        print(std_residuals[abs(std_residuals) > 3])

        predictions_test = model_ols.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
        print(f"\nTest RMSE for OLS in {country}: {rmse:.4f}")

        predictions = model_ols.predict(X)
        country_data['predicted_net_migration_ols'] = np.exp(predictions) - shift


        plt.figure(figsize=(8, 6))
        plt.scatter(predictions_test, y_test, label='Data points')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect fit')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'OLS Predicted vs Actual for {country}')
        plt.legend()
        plt.savefig(f'country_results/{country.replace(" ", "_")}_ols_plot.png')
        plt.close()
    except Exception as e:
        error_msg = f"Error for {country} OLS: Model fitting failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
    

    try:
        model_quad = sm.OLS(y_train, X_train).fit(cov_type='HC3') 
        models['Quadratic'] = {'model': model_quad, 'type': 'Quadratic'}
        print(f"\nQuadratic Model Summary for {country}:")
        print(model_quad.summary())
     
        residuals = model_quad.resid
        std_residuals = residuals / residuals.std()
        print(f"\nStandardized Residuals Check for Quadratic in {country} (absolute values > 3):")
        print(std_residuals[abs(std_residuals) > 3])
 
        predictions_test = model_quad.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
        print(f"\nTest RMSE for Quadratic in {country}: {rmse:.4f}")

        predictions = model_quad.predict(X)
        country_data['predicted_net_migration_quad'] = np.exp(predictions) - shift


        plt.figure(figsize=(8, 6))
        plt.scatter(predictions_test, y_test, label='Data points')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect fit')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'Quadratic Predicted vs Actual for {country}')
        plt.legend()
        plt.savefig(f'country_results/{country.replace(" ", "_")}_quad_plot.png')
        plt.close()
    except Exception as e:
        error_msg = f"Error for {country} Quadratic: Model fitting failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
    

    try:

        endog_vars = ['ln_gdp_centered', 'polstab_centered']
   
        instruments = ['ln_gdp_centered_lag1', 'polstab_centered_lag1']

        exog_vars = ['ln_gdp_centered_sq', 'ln_gini', 'ln_unemp', 'covid_2020_2021']
 
        X_2sls = country_data[exog_vars + endog_vars]
        Z_2sls = country_data[exog_vars + instruments]
        X_2sls = sm.add_constant(X_2sls)
        Z_2sls = sm.add_constant(Z_2sls)
        

        X_train_2sls, X_test_2sls, y_train_2sls, y_test_2sls = train_test_split(
            X_2sls, y, test_size=0.2, random_state=42
        )
        Z_train_2sls = Z_2sls.loc[X_train_2sls.index]
        

        model_2sls = IV2SLS(y_train_2sls, X_train_2sls, instrument=Z_train_2sls).fit()
        models['2SLS'] = {'model': model_2sls, 'type': '2SLS'}
        print(f"\n2SLS Model Summary for {country}:")
        print(model_2sls.summary())
        

        residuals = model_2sls.resid
        std_residuals = residuals / residuals.std()
        print(f"\nStandardized Residuals Check for 2SLS in {country} (absolute values > 3):")
        print(std_residuals[abs(std_residuals) > 3])

        predictions_test = model_2sls.predict(X_test_2sls)
        rmse = np.sqrt(mean_squared_error(y_test_2sls, predictions_test))
        print(f"\nTest RMSE for 2SLS in {country}: {rmse:.4f}")

        predictions = model_2sls.predict(X_2sls)
        country_data['predicted_net_migration_2sls'] = np.exp(predictions) - shift
        rmse = np.sqrt(mean_squared_error(y_test_2sls, predictions_test))
        print(f"\nTest RMSE for 2SLS in {country}: {rmse:.4f}")

        plt.figure(figsize=(8, 6))
        plt.scatter(predictions_test, y_test_2sls, label='Data points')
        plt.plot([y_test_2sls.min(), y_test_2sls.max()], [y_test_2sls.min(), y_test_2sls.max()], 'r--', label='Perfect fit')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'2SLS Predicted vs Actual for {country}')
        plt.legend()
        plt.savefig(f'country_results/{country.replace(" ", "_")}_2sls_plot.png')
        plt.close()
    except Exception as e:
        error_msg = f"Error for {country} 2SLS: Model fitting failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
    

    output_file = f'country_results/{country.replace(" ", "_")}_predictions.csv'
    country_data.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

log_file.close()
print("\nProcessing complete for all countries. Check 'country_results/error_log.txt' for details on any errors.")

In [ ]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5, random_state=42).fit(X_train.drop('const', axis=1), y_train)
print("Selected features:", X_train.columns[lasso.coef_ != 0])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(predictions_test, y_test)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Predicted vs Actual for {country}')
plt.show()

In [ ]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5, random_state=42).fit(X_train.drop('const', axis=1), y_train)
print("Selected features:", X_train.drop('const', axis=1).columns[lasso.coef_ != 0])

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
import os


data = pd.read_csv('migrationdata.csv')

if not os.path.exists('country_results'):
    os.makedirs('country_results')


log_file = open('country_results/error_log.txt', 'w')


data = data.sort_values(['country', 'year'])
data['covid_2020_2021'] = data['year'].isin([2020, 2021]).astype(int)
data['gdp_centered'] = data.groupby('country')['gdp'].transform(lambda x: x - x.mean())
data['ln_gdp_centered'] = np.log(data['gdp_centered'].abs() + 1) * np.sign(data['gdp_centered'])
data['ln_gdp_centered_sq'] = data['ln_gdp_centered'] ** 2
data['ln_gini'] = np.log(data['gini'] + 1)
data['ln_unemp'] = np.log(data['unemp'] + 1)
data['polstab_centered'] = data.groupby('country')['polstab'].transform(lambda x: x - x.mean())
data['ln_gdp_centered_lag1'] = data.groupby('country')['ln_gdp_centered'].shift(1)
data['polstab_centered_lag1'] = data.groupby('country')['polstab_centered'].shift(1)
min_net_migration = data['net_migration'].min()
shift = abs(min_net_migration) + 1 if min_net_migration <= 0 else 1
data['ln_net_migration'] = np.log(data['net_migration'] + shift)
data = data.dropna()


countries = data['country'].unique()

for country in countries:
    print(f"\nProcessing {country}...\n")
    country_data = data[data['country'] == country].copy()
    
    if len(country_data) < 10:
        error_msg = f"Error for {country}: Insufficient data points ({len(country_data)} < 10)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    
    predictors = ['ln_gdp_centered', 'ln_gdp_centered_sq', 'ln_gini', 'ln_unemp', 
                  'polstab_centered', 'ln_gdp_centered_lag1', 'polstab_centered_lag1', 
                  'covid_2020_2021']
    if country_data[predictors + ['ln_net_migration']].isna().any().any():
        error_msg = f"Error for {country}: Missing values in predictors or response\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    
    X = country_data[predictors]
    y = country_data['ln_net_migration']
    
    print(f"Correlation Matrix for {country}:")
    print(X.corr())
    
    print(f"\nNaN/Inf Check Before Model for {country}:")
    print(X.isna().sum())
    print(X.isin([np.inf, -np.inf]).sum())
    
    vif_data = pd.DataFrame()
    vif_data['variable'] = X.columns
    try:
        vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        print(f"\nVIF Results for {country}:")
        print(vif_data)
        if (vif_data['VIF'] > 10).any():
            print(f"Warning for {country}: High VIF detected, potential multicollinearity")
            log_file.write(f"Warning for {country}: High VIF detected, potential multicollinearity\n")
    except np.linalg.LinAlgError:
        error_msg = f"Error for {country}: VIF calculation failed (possible singular matrix)\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    
    X = sm.add_constant(X)
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    except ValueError as e:
        error_msg = f"Error for {country}: Train-test split failed ({str(e)})\n"
        print(error_msg)
        log_file.write(error_msg)
        continue
    

    try:
        if country == 'Germany':
            model = sm.RLM(y_train, X_train, M=sm.robust.norms.HuberT()).fit()
            model_type = "Huber Robust Regression"
        else:
            model = sm.GLSAR(y_train, X_train, rho=1).fit(cov_type='HC3')
            model_type = "Prais-Winsten"
    except (np.linalg.LinAlgError, ValueError) as e:
        error_msg = f"Warning for {country}: {model_type} failed ({str(e)}), falling back to OLS\n"
        print(error_msg)
        log_file.write(error_msg)
        model = sm.OLS(y_train, X_train).fit(cov_type='HC3')
        model_type = "OLS"
    
    print(f"\n{model_type} Model Summary for {country}:")
    print(model.summary())
    

    if model_type != "Huber Robust Regression" and hasattr(model, 'rsquared') and model.rsquared < 0.7:
        print(f"\nLow R² ({model.rsquared:.3f}) for {country}, trying LASSO...")
        try:
            lasso = LassoCV(cv=5, random_state=42).fit(X_train.drop('const', axis=1), y_train)
            selected_features = X_train.drop('const', axis=1).columns[lasso.coef_ != 0]
            print(f"Selected features by LASSO: {list(selected_features)}")
            predictions_test = lasso.predict(X_test.drop('const', axis=1))
            rmse_lasso = np.sqrt(mean_squared_error(y_test, predictions_test))
            print(f"LASSO Test RMSE for {country}: {rmse_lasso:.4f}")
            if rmse_lasso < np.sqrt(mean_squared_error(y_test, model.predict(X_test))):
                predictions = lasso.predict(X.drop('const', axis=1))
                model_type = "LASSO"
            else:
                predictions = model.predict(X)
        except Exception as e:
            error_msg = f"Error for {country}: LASSO failed ({str(e)})\n"
            print(error_msg)
            log_file.write(error_msg)
            predictions = model.predict(X)
    else:
        predictions = model.predict(X)
    
    residuals = y_train - model.predict(X_train) if model_type != "LASSO" else y_train - lasso.predict(X_train.drop('const', axis=1))
    std_residuals = residuals / residuals.std()
    print(f"\nStandardized Residuals Check for {country} (absolute values > 3):")
    print(std_residuals[abs(std_residuals) > 3])
    
    predictions_test = model.predict(X_test) if model_type != "LASSO" else lasso.predict(X_test.drop('const', axis=1))
    rmse = np.sqrt(mean_squared_error(y_test, predictions_test))
    print(f"\nTest RMSE for {country}: {rmse:.4f}")
    
    country_data['predicted_net_migration'] = np.exp(predictions) - shift
    output_file = f'country_results/{country.replace(" ", "_")}_prais.csv'
    country_data.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

log_file.close()
print("\nProcessing complete for all countries. Check 'country_results/error_log.txt' for details on any errors.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.ticker import MaxNLocator


plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("paper", font_scale=1.2)


output_dir = "country_plots"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


df = pd.read_csv("migrationdata.csv")


df['year'] = df['year'].astype(int)
df[['net_migration', 'gdp', 'gini', 'polstab', 'unemp']] = df[['net_migration', 'gdp', 'gini', 'polstab', 'unemp']].astype(float)


countries = df['country'].unique()


variables = ['net_migration', 'gdp', 'gini', 'polstab', 'unemp']
labels = ['Net Migration', 'GDP per Capita', 'Gini Coefficient', 'Political Stability', 'Unemployment Rate']

for country in countries:

    country_df = df[df['country'] == country].sort_values('year')

    fig, axes = plt.subplots(5, 1, figsize=(10, 12), sharex=True)
    fig.suptitle(f'Time Series Analysis for {country} (1986–2022)', fontsize=16, y=0.98)
    

    for i, (var, label) in enumerate(zip(variables, labels)):
        axes[i].plot(country_df['year'], country_df[var], marker='o', markersize=4, linewidth=1.5, color='tab:blue')
        axes[i].set_ylabel(label)
        axes[i].grid(True, linestyle='--', alpha=0.7)

        axes[i].xaxis.set_major_locator(MaxNLocator(integer=True))

        if i == 4:  
            axes[i].set_xlabel('Year')
            plt.xticks(rotation=45)
    

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    

    plt.savefig(os.path.join(output_dir, f'{country}_timeseries.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # correlation heatmap
    plt.figure(figsize=(8, 6))
    corr_matrix = country_df[variables].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, 
                square=True, fmt='.2f', annot_kws={'size': 10})
    plt.title(f'Correlation Matrix for {country} (1986–2022)', fontsize=14, pad=15)
    
    # save heatmap
    plt.savefig(os.path.join(output_dir, f'{country}_correlation_heatmap.png'), dpi=300, bbox_inches='tight')
    plt.close()

print(f"Plots saved in '{output_dir}' directory.")

Plots saved in 'country_plots' directory.
